In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import json

import pandas as pd

In [ ]:
import os

big_data = pd.DataFrame()
## find current folder root
root = os.getcwd()

all_files = glob.iglob('downloaded_configs/passivbot_v5.8.0/configs' + '/**/result.txt', recursive=True)


files = []
for file in all_files:
    if '\\old\\' not in file:
        files.append(file)

i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = os.path.abspath(filename.replace('result.txt', '../config.json'))
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())

        big_data = big_data.append(d, ignore_index=True)
big_data

In [ ]:
allowed_symbols = [
    "ETHUSDT", "BCHUSDT", "XRPUSDT", "EOSUSDT", "LTCUSDT", "TRXUSDT", "ETCUSDT", "LINKUSDT", "XLMUSDT", "ADAUSDT",
    "XMRUSDT", "DASHUSDT", "ZECUSDT", "XTZUSDT", "BNBUSDT", "ATOMUSDT", "NEOUSDT", "IOSTUSDT", "THETAUSDT", "ALGOUSDT",
    "ZILUSDT", "KNCUSDT", "COMPUSDT", "OMGUSDT", "DOGEUSDT", "KAVAUSDT", "BANDUSDT", "RLCUSDT", "WAVESUSDT", "SNXUSDT",
    "DOTUSDT", "CRVUSDT", "TRBUSDT", "RUNEUSDT", "SUSHIUSDT", "SRMUSDT", "EGLDUSDT", "FTMUSDT", "HNTUSDT", "RENUSDT",
    "KSMUSDT", "NEARUSDT", "FILUSDT", "RSRUSDT", "MATICUSDT", "OCEANUSDT", "CVCUSDT", "GRTUSDT", "1INCHUSDT", "CHZUSDT",
    "SANDUSDT", "BTSUSDT", "LITUSDT", "UNFIUSDT", "REEFUSDT", "MANAUSDT", "HBARUSDT", "ONEUSDT", "LINAUSDT", "STMXUSDT",
    "DENTUSDT", "HOTUSDT", "MTLUSDT", "SCUSDT", "1000SHIBUSDT"
]
#allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
    #& (big_data['long'] == True)
    #& (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    #& (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    #& (big_data['ebrl'] > .8)
    #& (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df.to_csv('5.8.csv')
df

In [ ]:
df.drop_duplicates(
    subset=['s',  'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file',  's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

In [ ]:
dl_proposed = dx[
    #(dx['lm'] == 'n') &
    (dx['bnkrl_%'] == 100) &
    (dx['bnkrs_%'] == 100) &
    (dx['ebrl'] > 0.9) &  #&
    (dx['ebrs'] > 0.9) &  #&
    # (dx['adgl_%'] > 0.04) 
    (dx['pastdl'] < 0.03) &
    (dx['pastds'] < 0.03) &  #
    (dx['maxsl'] < 10) &
    (dx['maxss'] < 10) &
    (dx['adgt_%'].notnull())
    ]
#dl_proposed = dl_proposed[['file','s','adgl_%', 'bnkrl_%', 'ebrl',  'maxsl', 'pamaxl','pastdl' ]]
dl_proposed